I NEED TO GET THE CORRECT RED SHIFT BEFORE I USE PROSPECTOR

In [18]:
from astroquery.sdss import SDSS
from astropy import coordinates as coords
import astropy.units as u

# Define the target parameters
ra = 158.883596183724
dec = 37.6495783499432
sdss_id = 1237662225669161128

# Create a SkyCoord object for the target coordinates
target_coord = coords.SkyCoord(ra, dec, unit=(u.deg, u.deg))

# Query the spectroscopic data around the target with a radius of 2 arcsec
result = SDSS.query_region(target_coord, radius=2*u.arcsec, spectro=True)

# Check if we got any results
if result is None or len(result) == 0:
    print("No spectroscopic data found for the target.")
else:
    # Filter the result by the provided SDSS object ID (objid column)
    mask = result['objid'] == sdss_id
    if mask.sum() == 0:
        print("No matching object found with the specified SDSS ID.")
    else:
        # Extract redshift and redshift error for the first matching row
        z = result[mask]['z'][0]
       
        print("Redshift (z):", z)
        


Redshift (z): 0.06922206


In [32]:
from astroquery.sdss import SDSS
from astropy import coordinates as coords
import astropy.units as u

# Define the target parameters
ra = 158.883596183724
dec = 37.6495783499432

# Create a SkyCoord object for the target coordinates
target_coord = coords.SkyCoord(ra, dec, unit=(u.deg, u.deg))

# Query the spectroscopic data around the target with a radius of 3 arcsec
result = SDSS.query_region(target_coord, radius=3*u.arcsec, spectro=True)

# Check if we got any results
if result is None or len(result) == 0:
    print("No spectroscopic data found for the target.")
else:
    # Assume the first entry is the desired object and extract its ID and redshift
    obj_id = result['objid'][0]
    z = result['z'][0]
    
    print("Object ID:", obj_id)
    print("Redshift (z):", z)


Object ID: 1237662225669161128
Redshift (z): 0.06922206


THIS CODE OUTPUTS THE DATA WITH ERROR

In [3]:
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy import units as u
from astroquery.sdss import SDSS
from astropy import coordinates as coords

# Read the CSV file
try:
    df = pd.read_csv("/home/jovyan/research/2.14.25/diego_data.csv")  # Replace with your actual file path
except FileNotFoundError:
    print("Error: CSV file not found. Please make sure the file exists in the correct directory.")
    exit()

# Iterate through the first 5 supernovae (or less if the file has fewer rows)
for i in range(min(5, len(df))):
    ztf_id = df['ZTFID'][i]  # Column name for the ZTF ID
    ra_dec_str = f"{df['RA'][i]} {df['Dec'][i]}"  # Combine RA and Dec into one string

    try:
        # Create a SkyCoord object from the RA/Dec string.
        # Assumes RA is in hourangle format and Dec in degrees (e.g. "12:34:56 +12:34:56")
        sn_coords = SkyCoord(ra_dec_str, unit=(u.hourangle, u.deg))
        
        search_radius = 60 * u.arcsec

        # Query SDSS DR17 for photometric information including magnitude errors for each band.
        photoobj_fields = [
            'objID', 'ra', 'dec', 'type', 'specObjID', 'z',
            'u', 'err_u', 'g', 'err_g', 'r', 'err_r', 'i', 'err_i'
        ]
        results = SDSS.query_region(
            sn_coords,
            radius=search_radius,
            data_release=17,
            photoobj_fields=photoobj_fields,
        )

        print(f"\nZTF ID: {ztf_id}")

        if results is not None and len(results) > 0:
            # Calculate separations between the input coordinate and each returned object
            distances = sn_coords.separation(
                SkyCoord(ra=results['ra'] * u.degree, dec=results['dec'] * u.degree, frame='icrs')
            )
            sorted_indices = distances.argsort()
            sorted_results = results[sorted_indices]

            # Filter the results to include only galaxies (type == 3)
            galaxies = sorted_results[sorted_results['type'] == 3]

            if len(galaxies) > 0:
                closest_galaxy = galaxies[0]
                print("Possible Host Galaxy:")
                print(f"  SDSS ID: {closest_galaxy['objID']}")
                print(f"  Type: {closest_galaxy['type']}")
                print(f"  RA: {closest_galaxy['ra']}")
                print(f"  Dec: {closest_galaxy['dec']}")
                print(f"  u: {closest_galaxy['u']} ± {closest_galaxy['err_u']}")
                print(f"  g: {closest_galaxy['g']} ± {closest_galaxy['err_g']}")
                print(f"  r: {closest_galaxy['r']} ± {closest_galaxy['err_r']}")
                print(f"  i: {closest_galaxy['i']} ± {closest_galaxy['err_i']}")
                print(f"  z (photometric): {closest_galaxy['z']}")
                print("CHECK THE WEBSITE TO CONFIRM")

                # Use the RA and Dec of the host galaxy to query for spectroscopic data
                host_ra = closest_galaxy['ra']
                host_dec = closest_galaxy['dec']

                # Create a SkyCoord object for the host galaxy
                target_coord = coords.SkyCoord(host_ra, host_dec, unit=(u.deg, u.deg))

                # Query the spectroscopic data around the target with a radius of 3 arcsec
                spec_result = SDSS.query_region(target_coord, radius=3*u.arcsec, spectro=True)

                if spec_result is None or len(spec_result) == 0:
                    print("No spectroscopic data found for the host galaxy.")
                else:
                    # Extract object ID, redshift, redshift error, and field for the first entry
                    spec_obj_id = spec_result['objid'][0]
                    spec_z = spec_result['z'][0]
                    
                    if 'zErr' in spec_result.colnames:
                        spec_z_err = spec_result['zErr'][0]
                    elif 'z_err' in spec_result.colnames:
                        spec_z_err = spec_result['z_err'][0]
                    else:
                        spec_z_err = None
                        print("Warning: Redshift error column not found in spectroscopic data.")

                    if 'field' in spec_result.colnames:
                        spec_field = spec_result['field'][0]
                    else:
                        spec_field = None
                        print("Warning: 'field' column not found in spectroscopic data.")

                    print("Spectroscopic Data:")
                    print(f"  Object ID: {spec_obj_id}")
                    print(f"  Redshift (z): {spec_z}")
                    print(f"  Redshift Error (zErr): {spec_z_err}")
                    print(f"  Field: {spec_field}")
            else:
                print("No galaxies found within the search radius.")
        else:
            print("No SDSS objects found within the search radius.")

    except (ValueError, AttributeError) as e:
        print(f"Error processing coordinates for ZTF ID {ztf_id}: {e}")
        continue  # Skip to the next supernova if there's an error



ZTF ID: ZTF17aabtvsy
Possible Host Galaxy:
  SDSS ID: 1237662225669161128
  Type: 3
  RA: 158.883596183724
  Dec: 37.6495783499432
  u: 17.92454 ± 0.03000162
  g: 16.04441 ± 0.003830316
  r: 15.19806 ± 0.002961723
  i: 14.7865 ± 0.002980798
  z (photometric): 14.47726
CHECK THE WEBSITE TO CONFIRM
Spectroscopic Data:
  Object ID: 1237662225669161128
  Redshift (z): 0.06922206
  Redshift Error (zErr): None
  Field: 238

ZTF ID: ZTF17aabvong
Possible Host Galaxy:
  SDSS ID: 1237678919681179839
  Type: 3
  RA: 31.2821072265924
  Dec: 11.2485784557871
  u: 16.86292 ± 0.01934663
  g: 15.56673 ± 0.003088403
  r: 14.88181 ± 0.002867637
  i: 14.50039 ± 0.002816194
  z (photometric): 14.19007
CHECK THE WEBSITE TO CONFIRM
No spectroscopic data found for the host galaxy.

ZTF ID: ZTF17aacldgo
No SDSS objects found within the search radius.

ZTF ID: ZTF17aadlxmv
Possible Host Galaxy:
  SDSS ID: 1237657628436005116
  Type: 3
  RA: 127.448173699413
  Dec: 33.9064715077163
  u: 17.73309 ± 0.02001226


THIS CODE PUTS ALL THE DATA IN A CSV

In [1]:
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy import units as u
from astroquery.sdss import SDSS
from astropy import coordinates as coords

# Read the CSV file
try:
    df = pd.read_csv("/home/jovyan/research/2.14.25/diego_data.csv")  # Replace with your actual file path
except FileNotFoundError:
    print("Error: CSV file not found. Please make sure the file exists in the correct directory.")
    exit()

# List to hold results for each supernova
output_rows = []

# Iterate through the first 5 supernovae (or less if the file has fewer rows)
for i in range(min(5, len(df))):
    ztf_id = df['ZTFID'][i]  # Column name for the ZTF ID
    ra_dec_str = f"{df['RA'][i]} {df['Dec'][i]}"  # Combine RA and Dec into one string

    try:
        # Create a SkyCoord object from the RA/Dec string.
        # Assumes RA is in hourangle format and Dec in degrees (e.g. "12:34:56 +12:34:56")
        sn_coords = SkyCoord(ra_dec_str, unit=(u.hourangle, u.deg))
        
        search_radius = 60 * u.arcsec

        # Query SDSS DR17 for photometric information including magnitude errors for each band.
        photoobj_fields = [
            'objID', 'ra', 'dec', 'type', 'specObjID', 'z',
            'u', 'err_u', 'g', 'err_g', 'r', 'err_r', 'i', 'err_i'
        ]
        photo_results = SDSS.query_region(
            sn_coords,
            radius=search_radius,
            data_release=17,
            photoobj_fields=photoobj_fields,
        )

        print(f"\nZTF ID: {ztf_id}")

        if photo_results is not None and len(photo_results) > 0:
            # Calculate separations between the input coordinate and each returned object
            distances = sn_coords.separation(
                SkyCoord(ra=photo_results['ra'] * u.degree, dec=photo_results['dec'] * u.degree, frame='icrs')
            )
            sorted_indices = distances.argsort()
            sorted_results = photo_results[sorted_indices]

            # Filter the results to include only galaxies (type == 3)
            galaxies = sorted_results[sorted_results['type'] == 3]

            if len(galaxies) > 0:
                closest_galaxy = galaxies[0]
                print("Possible Host Galaxy found.")

                # Use the RA and Dec of the host galaxy to query for spectroscopic data
                host_ra = closest_galaxy['ra']
                host_dec = closest_galaxy['dec']

                # Create a SkyCoord object for the host galaxy
                target_coord = coords.SkyCoord(host_ra, host_dec, unit=(u.deg, u.deg))

                # Query the spectroscopic data around the target with a radius of 3 arcsec
                spec_result = SDSS.query_region(target_coord, radius=3*u.arcsec, spectro=True)

                if spec_result is None or len(spec_result) == 0:
                    print("No spectroscopic data found for the host galaxy.")
                    spec_obj_id = None
                    spec_z = None
                    spec_z_err = None
                    spec_field = None
                else:
                    # Extract spectroscopic data for the first entry
                    spec_obj_id = spec_result['objid'][0]
                    spec_z = spec_result['z'][0]
                    
                    if 'zErr' in spec_result.colnames:
                        spec_z_err = spec_result['zErr'][0]
                    elif 'z_err' in spec_result.colnames:
                        spec_z_err = spec_result['z_err'][0]
                    else:
                        spec_z_err = None
                        print("Warning: Redshift error column not found in spectroscopic data.")

                    if 'field' in spec_result.colnames:
                        spec_field = spec_result['field'][0]
                    else:
                        spec_field = None
                        print("Warning: 'field' column not found in spectroscopic data.")

                # Create a dictionary for this entry
                row = {
                    'ZTFID': ztf_id,
                    'photo_objID': closest_galaxy['objID'],
                    'photo_type': closest_galaxy['type'],
                    'photo_ra': closest_galaxy['ra'],
                    'photo_dec': closest_galaxy['dec'],
                    'u': closest_galaxy['u'],
                    'err_u': closest_galaxy['err_u'],
                    'g': closest_galaxy['g'],
                    'err_g': closest_galaxy['err_g'],
                    'r': closest_galaxy['r'],
                    'err_r': closest_galaxy['err_r'],
                    'i': closest_galaxy['i'],
                    'err_i': closest_galaxy['err_i'],
                    'photo_z': closest_galaxy['z'],
                    'spec_objid': spec_obj_id,
                    'spec_z': spec_z,
                    'spec_z_err': spec_z_err,
                    'spec_field': spec_field
                }
                output_rows.append(row)
            else:
                print("No galaxies found within the search radius for this supernova.")
        else:
            print("No SDSS photometric objects found within the search radius for this supernova.")

    except (ValueError, AttributeError) as e:
        print(f"Error processing coordinates for ZTF ID {ztf_id}: {e}")
        continue  # Skip to the next supernova if there's an error

# Create a DataFrame from the collected data and save to CSV
if output_rows:
    output_df = pd.DataFrame(output_rows)
    output_df.to_csv("/home/jovyan/research/2.14.25/host_gal_data.csv", index=False)
    print("\nData successfully written to host_gal_data.csv")
else:
    print("\nNo host galaxy data was collected.")



ZTF ID: ZTF17aabtvsy
Possible Host Galaxy found.

ZTF ID: ZTF17aabvong
Possible Host Galaxy found.
No spectroscopic data found for the host galaxy.

ZTF ID: ZTF17aacldgo
No SDSS photometric objects found within the search radius for this supernova.

ZTF ID: ZTF17aadlxmv
Possible Host Galaxy found.

ZTF ID: ZTF18aaaonon
Possible Host Galaxy found.

Data successfully written to host_gal_data.csv


In [1]:
import prospect
print(prospect.__file__)


/home/jovyan/.local/lib/python3.11/site-packages/prospect/__init__.py


In [4]:
import pandas as pd
import numpy as np
from astropy.cosmology import Planck15 as cosmo

# Load the dataset
file_path = "/home/jovyan/research/2.14.25/host_gal_data.csv"  # Update the path if needed
df = pd.read_csv(file_path)

# Ensure we use spec_z as the correct redshift column
df["redshift"] = df["spec_z"]

# Remove rows where redshift is NaN, as we need it to compute luminosity distance
df = df.dropna(subset=["redshift"])

# Compute luminosity distance (in Mpc) using redshift
df["luminosity_distance"] = cosmo.luminosity_distance(df["redshift"]).value

# Compute distance modulus (DM = 5 * log10(DL) + 25)
df["distance_modulus"] = 5 * np.log10(df["luminosity_distance"]) + 25

# Compute absolute magnitudes in each band (M = m - DM)
bands = ["u", "g", "r", "i", "photo_z"]  # "photo_z" is actually the z-band magnitude
for band in bands:
    df[f"abs_mag_{band}"] = df[band] - df["distance_modulus"]

# Save results to CSV or print first few rows
output_path = "/home/jovyan/research/2.14.25/absolute_magnitudes.csv"
df.to_csv(output_path, index=False)
print(f"Results saved to {output_path}")

# Optionally print first few rows
print(df[["ZTFID", "redshift", "luminosity_distance", "distance_modulus"] + [f"abs_mag_{b}" for b in bands]].head())


Results saved to /home/jovyan/research/2.14.25/absolute_magnitudes.csv
          ZTFID  redshift  luminosity_distance  distance_modulus  abs_mag_u  \
0  ZTF17aabtvsy  0.069222           322.235352         37.540866 -19.616326   
2  ZTF17aadlxmv  0.061960           286.973720         37.289211 -19.556121   
3  ZTF18aaaonon  0.071419           332.969372         37.612021 -18.294941   

   abs_mag_g  abs_mag_r  abs_mag_i  abs_mag_photo_z  
0 -21.496456 -22.342806 -22.754366       -23.063606  
2 -21.080261 -21.796551 -22.179931       -22.472431  
3 -20.294861 -21.214571 -21.663911       -22.002271  


In [3]:
import pandas as pd
import numpy as np
from astropy.cosmology import Planck15 as cosmo
from astropy import units as u
from speclite import filters

# Load your dataset
file_path = "/home/jovyan/research/Current/host_gal_data.csv"
df = pd.read_csv(file_path)

# Clean up redshift
df["redshift"] = df["spec_z"]
df = df.dropna(subset=["redshift"])

# Luminosity distance and distance modulus
df["luminosity_distance"] = cosmo.luminosity_distance(df["redshift"]).value  # Mpc
df["distance_modulus"] = 5 * np.log10(df["luminosity_distance"]) + 25

# Load SDSS filters from speclite
sdss_filters = filters.load_filters('sdss2010-u', 'sdss2010-g', 'sdss2010-r', 'sdss2010-i', 'sdss2010-z')

bands = ["u", "g", "r", "i", "photo_z"]
filter_dict = dict(zip(bands, sdss_filters))

# Define a simple function to approximate K-correction assuming a flat spectrum (for demonstration)
def approximate_k_correction(filter_band, redshift):
    # This assumes a flat (fν constant) spectrum; adjust as needed for accurate science!
    k_corr = -2.5 * np.log10(1 + redshift)
    return k_corr

# Calculate K-corrections and absolute magnitudes
for band in bands:
    df[f'k_corr_{band}'] = df['redshift'].apply(lambda z: approximate_k_correction(filter_dict[band], z))
    df[f'rest_abs_mag_{band}'] = df[band] - df['distance_modulus'] - df[f'k_corr_{band}']

# Save output
output_path = "/home/jovyan/research/Current/restframe_absolute_magnitudes.csv"
df.to_csv(output_path, index=False)

print(f"Results saved to {output_path}")
print(df[['ZTFID', 'redshift', 'distance_modulus'] + [f'rest_abs_mag_{b}' for b in bands]].head())


Results saved to /home/jovyan/research/Current/restframe_absolute_magnitudes.csv
          ZTFID  redshift  distance_modulus  rest_abs_mag_u  rest_abs_mag_g  \
0  ZTF17aabtvsy  0.069222         37.540866      -19.543656      -21.423786   
2  ZTF17aadlxmv  0.061960         37.289211      -19.490850      -21.014990   
3  ZTF18aaaonon  0.071419         37.612021      -18.220043      -20.219963   

   rest_abs_mag_r  rest_abs_mag_i  rest_abs_mag_photo_z  
0      -22.270136      -22.681696            -22.990936  
2      -21.731280      -22.114660            -22.407160  
3      -21.139673      -21.589013            -21.927373  
